In [4]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# display multiple outputs, default is 'last_expr' (last expression)
# http://ipython.readthedocs.io/en/stable/config/options/terminal.html
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy as sa
import psycopg2 as pg

from psycopg2.extras import execute_values

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

Pull down all features

In [19]:
query = '''
    SELECT *
    FROM prod.features
    WHERE "Season Type" = 'NCAA Tourney' and
        holdout_s1 = 0 and
        holdout_s2 = 0
'''

conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

df = pd.read_sql_query(query, conn)

In [20]:
df.dtypes.value_counts()
df.describe().T

float64    15
int64      10
object      2
dtype: int64

,count,mean,std,min,25%,50%,75%,max
Season,1426.0,2.008050e+03,3.176268,2003.000000,2005.000000,2008.000000,2011.000000,2013.000000
DayNum,1426.0,1.390996e+02,4.218910,134.000000,136.000000,137.000000,139.000000,154.000000
Team,1426.0,1.293577e+03,104.185887,1102.000000,1210.000000,1281.000000,1393.000000,1462.000000
Opponent,1426.0,1.293577e+03,104.185887,1102.000000,1210.000000,1281.000000,1393.000000,1462.000000
Outcome,1426.0,5.000000e-01,0.500175,0.000000,0.000000,0.500000,1.000000,1.000000
Score,1426.0,6.894741e+01,12.066513,29.000000,61.000000,69.000000,77.000000,121.000000
OpponentScore,1426.0,6.894741e+01,12.066513,29.000000,61.000000,69.000000,77.000000,121.000000
NumOT,1426.0,7.433380e-02,0.311331,0.000000,0.000000,0.000000,0.000000,2.000000
holdout_s1,1426.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
holdout_s2,1426.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
